# Kinetic Energy Analysis for a Protein System
# 2023/08 completed by Yoyo@MYTLab
# 2024/07/22 updated by Heery@MYTLab

MYT*Lab* website: https://mytsai.cc

參考資料：
- [NAMD tutorial ver.2017](https://www.ks.uiuc.edu/Training/Tutorials/namd-index.html)

此教程僅供教學用途

All rights reserved.

### 當製作出欲模擬系統的pdb、psf、conf檔後即可接續以下步驟（若為練習則可直接使用附加檔案進行操作）

# A. 配置文件設置

### &nbsp;&nbsp;&nbsp;A.0 打開 `JupyterLab` 確認左方視窗中路徑為`Desktop/kinetic_energy_MD`(路徑視需求更改)
### &nbsp;&nbsp;&nbsp;A.1 於左方視窗中以滑鼠打開名為`ubq_wb_eq.conf` 的檔案（欲計算動能系統的Configuration File）
### &nbsp;&nbsp;&nbsp;A.2 於檔案中`dcdfreq`指令下方多加名為`velDCDfreq`的指令並於後方設定匯出步數（此步驟已填入）
### &nbsp;&nbsp;&nbsp;A.3 調整`dcdfreq`和`velDCDfreq`的匯出步數（為方便之後動能和位能的計算，匯出步數最好一致）
### &nbsp;&nbsp;&nbsp;A.4 確認其他設定和模擬步數設定皆完成，即可進行namd模擬(模擬步數設定預設為12500)

# B. 使用NAMD進行分子動力學模擬

### &nbsp;&nbsp;B.0 在JupyterLab在左方視窗中點擊 " `+` " → 在右方 `Launcher` 介面中點 `Other` 區塊中的 `Terminal` 鍵入:
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;B.0.(a)鍵入:
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(Mac)
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;cd ~/Desktop/kinetic_energy_MD/
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(Windows)
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;cd ~\Desktop\kinetic_energy_MD\ 
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;B.0.(b)再鍵入:
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(Mac)
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;~/Desktop/NAMD_Tutorial_2022/NAMD_mac/namd2 ./ubq_wb_eq.conf > ubq_wb_eq.log
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(Windows)
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;~\Desktop\NAMD_Tutorial_2022\NAMD_exe\namd2 .\ubq_wb_eq.conf > ubq_wb_eq.log
    
### &nbsp;&nbsp;B.1 確認左方視窗中產生名為`ubq_wb_eq.dcd`、`ubq_wb_eq.veldcd`、`ubq_wb_eq.log`的檔案

# C. 於VMD內用腳本進行動能計算

### &nbsp;&nbsp;C.0 打開 VMD ( 預設路徑應為: C:/Program Files (x86)/University of Illinois/VMD )
### &nbsp;&nbsp;C.1 點VMD Main的【`Extensions`】→【`Tk Console`】→ 鍵入:
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;cd ~/Desktop/kinetic_energy_MD/ (更改為目標路徑) 
### &nbsp;&nbsp;C.2 點VMD Main的【`Extensions`】→【`Tk Console`】→ 鍵入: 
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;source Get_all_atoms_velvector_mass_trajectory.tcl  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(此腳本自動匯入檔案ˋubq_wb.psfˋ和ˋubq_wb_eq.veldcdˋ後計算系統個別原子質量和速度向量,其會產生`my_atoms_mass_.txt`和`my_atoms_velocity_vector_.txt`兩個檔案)
### &nbsp;&nbsp;C.3 於【`Tk Console`】繼續鍵入:
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;source Get_atom_index_select.tcl  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(若欲更改原子選取種類,可從腳本 `Get_atom_index_select.tcl` 內更改選取原子種類和範圍)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(此腳本自動匯入檔案ˋubq_wb.psfˋ和ˋubq_wb_eq.dcdˋ後計算選取範圍原子的index,其會產生`my_atom_indexs_select.txt`檔案)

### &nbsp;&nbsp;備註: `Get_all_atoms_velvector_mass_trajectory.tcl`、`Get_atom_index_select.tcl`腳本執行會自動匯入 ubq_wb.psf 和 ubq_wb_eq.dcd 、ubq_wb_eq.veldcd 檔案,若使用不同的蛋白質模擬需更改腳本內容的匯入檔名部分

# D. 以 JupyterLab 進行動能計算

### &nbsp;&nbsp;D.1 確認 `JupyterLab` 左方視窗中路徑為 `Desktop/kinetic_energy_MD` 中 (路徑視需求更改)
### &nbsp;&nbsp;D.2 動能計算:
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;D.2.(a) 載入模組

In [ ]:
import re
import numpy as np

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;D.2.(b1~b3) 用 Python 讀取腳本產生文檔，將原子的`速度向量`、`質量`、`選取原子的index`數據整理成易於分析的數組

#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;D.2.(b1) 讀取 my_atoms_velocity_vector_.txt 文檔,並將檔案內`原子的速度向量`數據整理成二維陣列後以 `my_velocity_vector_all_F` 列表儲存

In [ ]:

file_path = "./my_atoms_velocity_vector_.txt"                                                #(指定了一個文件路徑 file_path，指向名為 my_atoms_velocity_vector_.txt 的文本文件)

my_velocity_vector_all_F = []                                                                #(創建一個空列表 my_velocity_vector_all_F，此列表將用於存取全部幀數的原子速度向量資訊)
              
with open(file_path, "r") as file: 
    for line in file: 
        velocity_vector_list_per_F = []                                                      #(創建一個空列表 velocity_vector_list_per_F，此列表將用於存取每個幀數下全部原子的速度向量資訊)
        match = re.findall(r"{(.*?)}", line)                                                 #(使用正則表達式匹配括號{}中的所有內容)
        velocity_vector = []                                                                 #(創建一個空列表 velocity_vector，此列表將用於存取原子的速度向量資訊)
        for m in match:                        
            velocity_vector.extend([float(x) for x in m.split()])                            #(將列表 velocity_vector 寫入原子速度向量資訊)
        velocity_vector_list_per_F.append(velocity_vector)                                   #(將列表 velocity_vector 資訊寫入列表 velocity_vector_list_per_F 中,類似於由多個原子組成一個幀數的架構)
        my_velocity_vector_all_F.append(np.array(velocity_vector_list_per_F).reshape(-1, 3)) #(將 velocity_vector_list_per_F 轉換為 numpy 陣列，並使用 .reshape(-1, 3) 將其重新排列為一個 N×3 的陣列，每一行對應一組三維坐標（x, y, z）,以實現一個原子速度向量的三個分量。並將這個 N×3 的陣列寫入 my_velocity_vector_all_F 列表中) 

my_velocity_vector_all_F
#(以列表存取 N×3 的陣列,以實現由多個原子組成一個幀數,並以多個幀數組成一個個連續軌跡的架構)       

#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;D.2.(b-2) 讀取 my_atoms_mass_.txt 文檔,並將檔案內`原子的質量`數據整理成陣列後以 `my_mass_all_F` 列表儲存

In [ ]:

file_path = "./my_atoms_mass_.txt"                           #(指定了一個文件路徑 file_path，指向名為 my_atoms_mass_.txt 的文本文件)

my_mass_all_F = []                                           #(創建一個空列表 my_mass_all_F，此列表將用於存取全部幀數的原子質量資訊)

with open(file_path, "r") as file: 
    for line in file:  
        index_str = line.split(": ")[1]  
        mass = [float(num) for num in index_str.split(" ")]  #(將原子質量寫入 mass,其包含一個幀數下的個別原子質量)
        mass_array = np.array(mass)                          #(將 mass 轉換為 numpy 陣列後,寫入 mass_array)
        my_mass_all_F.append(mass_array)                     #(將 mass_array 再寫入 my_mass_all_F 列表儲存)
        
my_mass_all_F 

#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;D.2.(b-3) 讀取 my_atom_indexs_select.txt 文檔,並將檔案內`選取原子的 index` 數據整理成陣列後以 `my_select_atom_index_all_F` 列表儲存

In [ ]:

file_path = "./my_atom_indexs_select.txt"                      #(指定了一個文件路徑 file_path，指向名為 my_atom_indexs_select.txt 的文本文件)

my_select_atom_index_all_F = []                                #(創建一個空列表 my_select_atom_index_all_F，此列表將用於存取全部幀數的原子index資訊)

with open(file_path, "r") as file:  
    for line in file:  
        index_str = line.split(": ")[1] 
        index = [int(num) for num in index_str.split(" ")]     #(將原子 index 資訊寫入 index ,其包含一個幀數下的個別原子index)
        index_array = np.array(index)                          #(將 index 轉換為 numpy 陣列後,寫入 index_array)
        my_select_atom_index_all_F.append(index_array)         #(將 index_array 再寫入 my_select_atom_index_all_F 列表儲存)
        
my_select_atom_index_all_F

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;D.2.(c) 分別整合出參照選取原子 index 的速度向量以及質量數據,並儲存於`select_velocity_vectors`、`select_mass`列表中

In [ ]:

select_velocity_vectors = []                                           #(創建兩個空列表 select_velocity_vectors 和 、select_mass，select_velocity_vectors 用於存儲選定原子的速度向量數據，select_mass 用於存儲這些原子的質量數據)
select_mass = []

for frame_i, atom_select in enumerate(my_select_atom_index_all_F):     #(使用 enumerate 函數遍歷 my_select_atom_index_all_F 列表內陣列，其中 frame_i 是幀的索引，atom_select 是當前幀中選定的原子索引)
    my_select_vector = my_velocity_vector_all_F[frame_i][atom_select]  #(根據當前幀的索引 frame_i 和當前幀中選定的原子索引陣列 atom_select 從 my_velocity_vector_all_F 提取出對應的原子的速度向量，並將其存儲在 my_select_vector 中)
    select_velocity_vectors.append(my_select_vector)                   #(將 my_select_vector（即選定原子的坐標數據）添加到 select_velocity_vectors 列表中)
    my_select_mass = my_mass_all_F[0][atom_select]                     #(從 my_mass_all_F 中提取第一幀的質量數據，並使用 atom_select 中的原子索引來選擇對應的原子質量數據，並將其存儲在 my_select_mass 中)
    select_mass.append(my_select_mass)                                 #(將 my_select_mass（即選定原子的質量數據）添加到 select_mass 列表中)

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;D.2.(d1~d3) 使用列表`select_velocity_vectors`、`select_mass` 中速度以及原子質量資訊完成動能計算：

#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;D.2.(d-1) 匯出檔案與布林值切換說明

 | 使用 `select_mass`、`select_velocity_vectors` 資訊 |

- set `get_atoms_Energy_and_mass` == Set `True` for `Maxwell-Boltzmann Energy Distribution` analysis

- return : `energy_list`, `atoms_energy_list_per_F`, `atoms_mass_list_per_F`


> `energy_list` : Total kinetic energy  perframe

> `atoms_energy_list_per_F` : [ [Array of the each atom's (After selection) kinetic energy on frame i], .....]

> `atoms_mass_list_per_F` : [ [Sum of the each atom's (After selection) mass on frame i], .....]

#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;D.2.(d-2) 定義函數以進行動能計算和數據處理(可產生 `energy_list`、`atoms_energy_list_per_F`、`atoms_mass_list_per_F` 三種檔案)

In [ ]:
#(定義了一個名為 kinetic_energy 的函數，接受三個參數：mass_list_per_frame(每幀中每個原子的質量)、velocity_vectors_per_frame(每幀中每個原子的速度向量)、get_atoms_Energy_and_mass(布林值，決定是否返回每個原子的動能和質量總和（默認為 False）)
def kinetic_energy(mass_list_per_frame, velocity_vectors_per_frame, get_atoms_Energy_and_mass=False):
    energy_list = []                                #(用於儲存每幀的總動能)
    atoms_energy_list_per_F = []                    #(用於儲存每幀中每個原子的動能)
    atoms_mass_list_per_F = []                      #(用於儲存每幀中原子的總質量)
    
    for frame_i, atoms_vectors_in_f in enumerate(velocity_vectors_per_frame):  #(遍歷 velocity_vectors_per_frame 中的每一幀，其中 frame_i 是幀的索引，atoms_vectors_in_f 是該幀中所有原子的速度向量)
        energy_frame_i = 0  
        atoms_energy_frame_i = []  
        atoms_mass_frame_i = 0      
        
        for atom_i, atom_i_vector in enumerate(atoms_vectors_in_f):            #(遍歷 atoms_vectors_in_f 中的每個原子，其中  atom_i 是原子的索引，atom_i_vector 是該原子的速度向量)
            atom_i_mass = mass_list_per_frame[frame_i][atom_i]                 #(從 mass_list_per_frame 中獲取與速度向量相對應的 [frame_i][atom_i]的原子質量 )
            atom_i_v_dot = np.dot(atom_i_vector, atom_i_vector)                #(使用 np.dot 計算速度向量的點積，即 v**2(速率平方))             
            atom_i_energy = 0.5 * atom_i_mass * atom_i_v_dot                   #(根據公式 EK = 0.5*m*v**2 計算動能)
            energy_frame_i += atom_i_energy                                    #(將原子的動能累加到 energy_frame_i 中)
            
            if get_atoms_Energy_and_mass:                                      #(如果 get_atoms_Energy_and_mass 為 True，將該原子的動能依序寫入到 atoms_energy_frame_i 列表中，並將質量累加到 atoms_mass_frame_i 中)
                atoms_energy_frame_i.append(atom_i_energy)  
                atoms_mass_frame_i += atom_i_mass  
                
        if get_atoms_Energy_and_mass:                                          #(如果 get_atoms_Energy_and_mass 為 True，將該幀的原子動能列表轉換為 NumPy 陣列，並將其寫入到 atoms_energy_list_per_F 中。將該幀的原子總質量寫入到 atoms_mass_list_per_F 中)
            atoms_energy_list_per_F.append(np.array(atoms_energy_frame_i))  
            atoms_mass_list_per_F.append(atoms_mass_frame_i) 
            
        print(f"Frame {frame_i} kinetic energy :  {energy_frame_i}")           #(打印該幀的總動能)
        energy_list.append(energy_frame_i)                                     #(將該幀的總動能添加到 energy_list 中)
        
    if not get_atoms_Energy_and_mass:                                          #(根據 get_atoms_Energy_and_mass 的值返回結果：如果為 False，僅返回 energy_list。如果為 True，返回三個列表：energy_list（每幀的總動能）、atoms_energy_list_per_F（每幀中每個原子的動能）、atoms_mass_list_per_F（每幀中原子的總質量）
        return energy_list
    else:
        return energy_list, atoms_energy_list_per_F, atoms_mass_list_per_F
    

#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;D.2.(d-3) 使用 `kinetic_energy 函數`計算 `atoms_energy_list_per_F (每幀中每個原子的動能)`,以進行 `Maxwell-Boltzmann Energy Distribution` 分析

In [ ]:
#(當 get_atoms_Energy_and_mass=True 時，其返回的列表 every_atoms_E_per_F(第二部分) 即為每幀中每個原子的動能)
total_E_per_F, every_atoms_E_per_F ,  every_atoms_M_per_F= kinetic_energy(select_mass, select_velocity_vectors, get_atoms_Energy_and_mass=True)

# E. Maxwell-Boltzmann Energy Distribution

### &nbsp;&nbsp;E.0 載入模組

In [ ]:
import math
import matplotlib.pyplot as plt
import seaborn as sns  

### &nbsp;&nbsp;E.1 定義 "Maxwell-Boltzmann distribution for kinetic energy" 公式

In [ ]:
def f_ek(Ek,kBT):
    f_ek_value = 2/(math.pi**0.5)*(1/(kBT**1.5))*(Ek**0.5)*math.exp(-Ek/kBT)    #(0.00198657是玻茲曼常數kBT，單位為 kcal/(mol*K))
    return f_ek_value

### &nbsp;&nbsp;E.2 使用 `f_ek 函数`計算指定幀數和溫度下每個原子的能量分布機率密度，並將结果儲存在 `data` 列表中

In [ ]:
data = [] 
Frame_index = 24    #(設置要處理的幀索引)
temperature = 280   #(設置溫度值（單位：開爾文 K）)

every_atoms_E_in_F_i = every_atoms_E_per_F[Frame_index]  #(從每幀的原子動能數據中提取指定幀的動能數據)

for i in every_atoms_E_in_F_i:                           #(遍歷指定幀的每個原子動能)
    result = f_ek(i, 0.00198657 * temperature)           #(調用函數 f_ek 計算結果，0.00198657是玻茲曼常數kBT，單位為 kcal/(mol*K))
    data.append(result)                                  # 將計算結果添加到 data 列表中

### &nbsp;&nbsp;E.3 將原子動能和對應的 Maxwell-Boltzmann 分布機率密度值结合起来，並產生橫坐標x(根據原子動能由小到大排序)的數組;縱坐標y(對應的Maxwell-Boltzmann分布機率密度值)的陣列

In [ ]:
# 將之前計算的結果轉換為NumPy陣列
y_array = np.array(data)

# 使用 np.c_ 函數將每個原子的動能數據和計算結果縱向拼接為新的數據集
# new_data 的每一行包含兩個元素：[原子動能, Maxwell-Boltzmann distribution計算結果]
new_data = np.c_[every_atoms_E_in_F_i, y_array]

# 根據原子動能對 new_data 進行排序,data_index 是按照 new_data 中第一列（原子動能）排序後的索引
data_index = np.argsort(new_data[:, 0])

# 根據排序索引 data_index 提取排序後的動能數據和計算結果
# x 是排序後的原子動能數據; y 是排序後的計算結果
x = new_data[:, 0][data_index]
y = new_data[:, 1][data_index]
x

### &nbsp;&nbsp;E.4 繪出包含兩個數據系列的圖形：散點圖，展示了原子動能的核密度分布；另一个是擬合後的 Maxwell-Boltzmann 分布曲線

In [ ]:
#(創建一個新的 Figure，用於繪製圖形，並指定圖形的尺寸大小為(15, 8))
fig = plt.figure()
plt.style.use('classic')
plt.figure(figsize=(15, 8))

#(設定圖形的標題和軸標籤，並調整字體大小)
plt.title("Maxwell-Boltzmann Energy Distribution", fontsize=20)
plt.ylabel("Relative Frequency", fontsize=20)
plt.xlabel("Energy (kcal/mol)", fontsize=20)

# 核密度估计 (KDE): Seaborn 的 kdeplot 函数會根據 every_atoms_E_in_F_i (每幀的原子動能)的數據點繪製出能量分布的散點圖。
sns.kdeplot(every_atoms_E_in_F_i, linestyle='', marker='o', color="red", 
            label="Our data (Unknown K)", linewidth=0)  # linestyle='' 和 linewidth=0

# 繪製我們擬合後的能量分佈曲線，設置 linewidth 參數以加粗曲線
plt.plot(x, y, label=f"Our data after fit kBT to {temperature} K", linewidth=2)

plt.legend()
plt.show()
